# Import libraries

In [2]:
import shap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif,RFE
from sklearn.model_selection import train_test_split, GridSearchCV,RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, classification_report, roc_auc_score, roc_curve,precision_recall_curve

# Read in data

In [3]:
# JIVE dataframes
cbcl_jive = pd.read_csv('cbcl.jive.sc.fc.out.csv') # [463 rows x 116 columns]> [drop first c]
cbcl_jive_embed = pd.read_csv('cbcl.jive.embed.csv') # [463 rows x 197 columns]> [drop first c]
cbcl_df = pd.read_csv('cbcl.sc.match.csv') # [463 rows x 3748 columns]>
cbcl_label = cbcl_df['CBCL'] # Name: CBCL, Length: 463, dtype: int64>

# Embedding dataframes
cbcl_fc_embed = pd.read_csv('cbcl_fc_node2vec_32embeddings_20wl.csv')# [463 rows x 3200 columns]>
cbcl_sc_embed = pd.read_csv('cbcl_node2vec_32embeddings_20wl.csv') # [602 rows x 2784 columns]>

# Vectorized dataframes
cbcl_fc_df = pd.read_csv('updated_flattened_fc_matrices_level_150.csv') # 463 rows × 4952 columns
cbcl_fc = cbcl_fc_df.filter(like='feature') # [463 rows x 4950 columns]>
cbcl_fc_label = cbcl_fc_df['CBCL'] # Name: CBCL, Length: 463, dtype: float64>

cbcl_sc_df = pd.read_csv('merged_dataset_cbcl.csv') # 602 rows × 3746 columns  
cbcl_sc = cbcl_sc_df.filter(like='V') # [602 rows x 3741 columns]>
cbcl_sc_label = cbcl_sc_df['CBCL'] # Name: CBCL, Length: 602, dtype: int64>

# Helpers

In [6]:
def find_best_threshold(y_true, y_prob):
    precision, recall, thresholds = precision_recall_curve(y_true, y_prob)
    f1_scores = 2 * (precision * recall) / (precision + recall)
    best_threshold = thresholds[np.argmax(f1_scores)]
    return best_threshold

In [18]:
def shap_feature_selection(X_train, y_train, k_best=10, random_state=42):
    # Train a preliminary XGBoost model
    model = XGBClassifier(random_state=random_state, eval_metric='logloss')
    model.fit(X_train, y_train)

    # Calculate SHAP values
    explainer = shap.Explainer(model, X_train)
    shap_values = explainer(X_train)

    # Get mean absolute SHAP values for each feature
    shap_importance = np.abs(shap_values.values).mean(axis=0)
    shap_importance_df = pd.DataFrame({
        'feature': X_train.columns,
        'shap_importance': shap_importance
    })

    # Select top k features based on SHAP values
    top_features = shap_importance_df.sort_values(by='shap_importance', ascending=False).head(k_best)['feature'].values
    return X_train[top_features], top_features


In [19]:
def xgboost_shap_4jive(X, y, k_best=10, test_size=0.4, random_state=42, n_splits=5, n_repeats=3):
    # Ensure all data is numeric
    X = X.apply(pd.to_numeric, errors='coerce')
    y = pd.to_numeric(y, errors='coerce')

    # Drop any rows with NaN values resulting from the coercion
    X.dropna(inplace=True)
    y = y[y.index.isin(X.index)]

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Feature selection using SHAP values
    X_train_selected, top_features = shap_feature_selection(X_train, y_train, k_best=k_best, random_state=random_state)
    X_test_selected = X_test[top_features]

    # Cross-validation and training for XGBoost
    xgb = XGBClassifier(random_state=random_state, eval_metric='logloss')
    xgb_param_grid = {
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 4, 5],
        'min_child_weight': [1, 2, 4],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }

    # Repeated Stratified K-Fold Cross Validation
    cv = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=random_state)

    # Perform cross-validation
    grid_search = GridSearchCV(xgb, xgb_param_grid, cv=cv, scoring='roc_auc', n_jobs=-1)
    grid_search.fit(X_train_selected, y_train)

    # Get the best estimator
    best_pipeline = grid_search.best_estimator_

    # Predict probabilities on the training set
    y_train_prob = best_pipeline.predict_proba(X_train_selected)[:, 1]

    # Find the best threshold based on the training set
    best_threshold = find_best_threshold(y_train, y_train_prob)

    # Predict probabilities on the test set
    y_test_prob = best_pipeline.predict_proba(X_test_selected)[:, 1]

    # Apply the best threshold to the test set
    y_pred_xgb = (y_test_prob >= best_threshold).astype(int)

    # Evaluate the model
    accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
    report_xgb = classification_report(y_test, y_pred_xgb)
    auc_xgb = roc_auc_score(y_test, y_test_prob)

    return best_pipeline, grid_search.best_params_, accuracy_xgb, auc_xgb, report_xgb, best_threshold


In [22]:
def xgboost_shap(X, y, n_components=50, k_best=10, test_size=0.4, random_state=42, n_splits=5, n_repeats=3):
    # Ensure all data is numeric
    X = X.apply(pd.to_numeric, errors='coerce')
    y = pd.to_numeric(y, errors='coerce')

    # Drop any rows with NaN values resulting from the coercion
    X.dropna(inplace=True)
    y = y[y.index.isin(X.index)]

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, shuffle=True)

    # Constant Feature Elimination (CFE)
    cfe = VarianceThreshold()
    X_train_cfe = cfe.fit_transform(X_train)
    X_test_cfe = cfe.transform(X_test)

    # Perform PCA
    pca = PCA(n_components=n_components)
    X_train_pca = pca.fit_transform(X_train_cfe)
    X_test_pca = pca.transform(X_test_cfe)

    # Feature selection using SHAP values
    X_train_selected, top_features = shap_feature_selection(pd.DataFrame(X_train_pca), y_train, k_best=k_best, random_state=random_state)
    X_test_selected = pd.DataFrame(X_test_pca)[top_features]

    # Cross-validation and training for XGBoost
    xgb = XGBClassifier(random_state=random_state, eval_metric='logloss')
    xgb_param_grid = {
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 4, 5],
        'min_child_weight': [1, 2, 4],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }

    # Repeated Stratified K-Fold Cross Validation
    cv = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=random_state)

    # Perform cross-validation
    grid_search = GridSearchCV(xgb, xgb_param_grid, cv=cv, scoring='roc_auc', n_jobs=-1)
    grid_search.fit(X_train_selected, y_train)

    # Get the best estimator
    best_pipeline = grid_search.best_estimator_

    # Predict probabilities on the training set
    y_train_prob = best_pipeline.predict_proba(X_train_selected)[:, 1]

    # Find the best threshold based on the training set
    best_threshold = find_best_threshold(y_train, y_train_prob)

    # Predict probabilities on the test set
    y_test_prob = best_pipeline.predict_proba(X_test_selected)[:, 1]

    # Apply the best threshold to the test set
    y_pred_xgb = (y_test_prob >= best_threshold).astype(int)

    # Evaluate the model
    accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
    report_xgb = classification_report(y_test, y_pred_xgb)
    auc_xgb = roc_auc_score(y_test, y_test_prob)

    return best_pipeline, grid_search.best_params_, accuracy_xgb, auc_xgb, report_xgb, best_threshold


# Main Codes

## [JIVE SC&FC]

In [9]:
best_pipeline, best_params, accuracy, auc, report, best_threshold = xgboost_shap_4jive(cbcl_jive.iloc[: , 1:], cbcl_label)
print("Best Parameters:", best_params)
print("Accuracy:", accuracy)
print("AUC:", auc)
print("Classification Report:\n", report)
print("Best Threshold:", best_threshold)

Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 4, 'min_child_weight': 4, 'n_estimators': 200, 'subsample': 1.0}
Accuracy: 0.553763440860215
AUC: 0.5587894248608535
Classification Report:
               precision    recall  f1-score   support

           0       0.56      0.77      0.64        98
           1       0.55      0.32      0.40        88

    accuracy                           0.55       186
   macro avg       0.55      0.54      0.52       186
weighted avg       0.55      0.55      0.53       186

Best Threshold: 0.7102117


## [JIVE Embedding SC&FC]

In [32]:
best_pipeline, best_params, accuracy, auc, report, best_threshold = xgboost_shap_4jive(cbcl_jive_embed.iloc[: , 1:], cbcl_label)
print("Best Parameters:", best_params)
print("Accuracy:", accuracy)
print("AUC:", auc)
print("Classification Report:\n", report)
print("Best Threshold:", best_threshold)

Best Parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.2, 'max_depth': 4, 'min_child_weight': 4, 'n_estimators': 200, 'subsample': 0.8}
Accuracy: 0.532258064516129
AUC: 0.5349025974025975
Classification Report:
               precision    recall  f1-score   support

           0       0.55      0.65      0.60        98
           1       0.51      0.40      0.45        88

    accuracy                           0.53       186
   macro avg       0.53      0.53      0.52       186
weighted avg       0.53      0.53      0.52       186

Best Threshold: 0.6069401


## [SC]

In [23]:
best_pipeline, best_params, accuracy, auc, report, best_threshold = xgboost_shap(cbcl_sc, cbcl_sc_label)
print("Best Parameters:", best_params)
print("Accuracy:", accuracy)
print("AUC:", auc)
print("Classification Report:\n", report)
print("Best Threshold:", best_threshold)

Best Parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.01, 'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 200, 'subsample': 0.8}
Accuracy: 0.5601659751037344
AUC: 0.5178374655647383
Classification Report:
               precision    recall  f1-score   support

           0       0.56      0.61      0.58       121
           1       0.56      0.51      0.54       120

    accuracy                           0.56       241
   macro avg       0.56      0.56      0.56       241
weighted avg       0.56      0.56      0.56       241

Best Threshold: 0.4620566


## [SC Embedding]

In [ ]:
best_pipeline, best_params, accuracy, auc, report, best_threshold = xgboost_shap(cbcl_sc_embed, cbcl_sc_label)
print("Best Parameters:", best_params)
print("Accuracy:", accuracy)
print("AUC:", auc)
print("Classification Report:\n", report)
print("Best Threshold:", best_threshold)

## [FC]

In [34]:
best_pipeline, best_params, accuracy, auc, report, best_threshold = xgboost_shap(cbcl_fc, cbcl_fc_label)
print("Best Parameters:", best_params)
print("Accuracy:", accuracy)
print("AUC:", auc)
print("Classification Report:\n", report)
print("Best Threshold:", best_threshold)

Best Parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 2, 'n_estimators': 200, 'subsample': 0.8}
Accuracy: 0.5
AUC: 0.5612858464384829
Classification Report:
               precision    recall  f1-score   support

         0.0       0.50      0.65      0.56        92
         1.0       0.51      0.35      0.42        94

    accuracy                           0.50       186
   macro avg       0.50      0.50      0.49       186
weighted avg       0.50      0.50      0.49       186

Best Threshold: 0.59834164


## [FC Embedding]

In [27]:
best_pipeline, best_params, accuracy, auc, report, best_threshold = xgboost_shap(cbcl_fc_embed, cbcl_fc_label)
print("Best Parameters:", best_params)
print("Accuracy:", accuracy)
print("AUC:", auc)
print("Classification Report:\n", report)
print("Best Threshold:", best_threshold)

Best Parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 0.8}
Accuracy: 0.553763440860215
AUC: 0.5254394079555966
Classification Report:
               precision    recall  f1-score   support

         0.0       0.53      0.78      0.63        92
         1.0       0.61      0.33      0.43        94

    accuracy                           0.55       186
   macro avg       0.57      0.56      0.53       186
weighted avg       0.57      0.55      0.53       186

Best Threshold: 0.5371731
